<a href="https://colab.research.google.com/github/ViMasHub/Test_Kontur/blob/main/Test_Kasperskiy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Загрузка библиотек

In [ ]:
import pandas as pd
import numpy as np
import re

import requests
from bs4 import BeautifulSoup

#Загрузка данных


In [ ]:
pd_test = pd.read_excel('/content/Test_check.xlsx')
pd_test.head()

,URL,lib_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп..."


#Предобработка данных


In [ ]:
# Установка библиотеки pymorphy3

!pip install pymorphy3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 65.5 MB/s eta 0:00:00


In [ ]:
# Загрузка библиотек

import nltk
import pymorphy3
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
# Загружаем набор данных для токенизатора
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

stop_words = stopwords.words('russian')
punctuation_marks = ['"','!', ',', '(', ')', ':', '-', '—', '?', '.', '..', '...']
morph = pymorphy3.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
len(stop_words)

151

In [ ]:
# Функция предобработки тескта
def preprocess (text, stop_words, punctuation_marks, morph):

  no_num_text = re.sub(r'\d', '', text)

  tokens = word_tokenize(no_num_text.lower())

  preprocessed_text = []

  for token in tokens:

    if token not in punctuation_marks:

      lemma = morph.parse(token)[0].normal_form

      if lemma not in stop_words:

        preprocessed_text.append(lemma)

  return preprocessed_text

In [ ]:
pd_test['pre_lib_text'] = pd_test.apply(lambda row: preprocess(row['lib_text'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
pd_test.head()

,URL,lib_text,pre_lib_text
0,https://expert.ru/ekonomika/vygodna-li-rossii-...,Серьезнее других от введения западных санкций ...,"[серьёзный, другой, введение, западный, санкци..."
1,https://ria.ru/20250311/klyuchevaya_stavka-196...,"МОСКВА, 14 фев — РИА Новости Ключевая процентн...","[москва, фев, риа, новость, ключевой, процентн..."
2,https://expert.ru/mnenie/denis-manturov-gosuda...,Со следующего года начнется реализация 12 мега...,"[следующий, год, начаться, реализация, мегапро..."
3,https://ria.ru/20250304/kredity-2003042476.html,Самозапрет на кредиты - это новая возможность ...,"[самозапрет, кредит, это, новый, возможность, ..."
4,https://lenta.ru/brief/2025/03/11/green/,"Заходя в магазин, вы часто замечали зеленые уп...","[заходить, магазин, часто, замечать, зелёный, ..."


#Парсинг эталонов

In [ ]:
postList = [] # список для извлекаемых данных


fullData = requests.get('https://expert.ru/ekonomika/vygodna-li-rossii-otmena-sanktsiy-ssha/') # запрос

soup = BeautifulSoup(fullData.text, 'html5lib') # парсинг страницы сайта

postList = soup.find_all('p')

postList



[<p>
 	 Серьезнее других от введения западных санкций пострадали отрасли, зависимые от импорта, — это нефтегаз и связанные с ним поставщики энергетического оборудования, которых отлучили от сервисных услуг и западных технологий добычи, а также машиностроение, заявил «Эксперту» директор Института народнохозяйственного прогнозирования РАН Александр Широв.
 </p>,
 <p>
 	 Директор по экономической политике НИУ ВШЭ Юрий Симачев добавил к этому списку ряд других отраслей: базовая электроника, отдельные направления IT (где, как он заметил, сейчас замещение идет достаточно эффективно), авиастроение, фармацевтика, а также отчасти легкая промышленность, где ситуация стабилизировалась после налаженных поставок из нейтральных стран.
 </p>,
 <p>
 	 В марте 2022 г. аналитики ЦМАКП, изучив зависимость России от импорта, прогнозировали, что сильнее всего санкции навредят фармацевтике (48,2% конечного потребления попало под санкции на март 2022 г.), сфере химических веществ и продуктов (44,7%), а также